## Init model

In [1]:
import warnings
warnings.filterwarnings('ignore')
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
%matplotlib inline  
from models.binary_model.binary_model import BinaryModel
from models.ind_model.ind_model import IndModel
from models.multi_model.multi_model import MultiModel
 
import pickle


mags = ["g_mag",  "r_mag", "i_mag", "z_mag", "y_mag",
        "W1_mag", "W2_mag",
        "J_mag", "K_mag", "H_mag"]


# mags = ["g_mag",  "r_mag", "i_mag", "z_mag", "y_mag", "u_mag",
#         "W1_mag", "W2_mag", "W3_mag", "W4_mag",
#         "J_mag", "K_mag", "H_mag",
#         "NUV_mag", "FUV_mag"]

# model = MultiModel(
#        folds = 40,
#        min_class_size = 40, 
# #        nb= True,
# #        max_class_size = 80,
# #        class_labels = ["Unspecified Ia",  "Unspecified II"], # "Unspecified II", "Ia-91bg", "TDE"],
# #        priors = [0.65, 0.36, 0.01, 0.005],
#        transform_features = True,
#        cols = mags,
#         )

# model.run_density_analysis()
# model.run_model()
# model.visualize_data()

## Load previous runs

In [2]:
# Rerun performance visualizations on saved output of model. 
def load_prev_exp(expnum, model):
    pickle_dir = "/Users/marina/Documents/PhD/research/astro_research/experiments/"+ expnum + "/"

    # with open(pickle_dir + 'density_results.pickle', 'rb') as handle:
    #     density_results = pickle.load(handle)  

    with open(pickle_dir + 'results.pickle', 'rb') as handle:
        results = pickle.load(handle)
    model.results = results

    with open(pickle_dir + 'y.pickle', 'rb') as handle:
        y = pickle.load(handle)    
    model.y = y
    return model



## Plot example outputs

In [ ]:
import random
import numpy as np

def plot_sample(model, priors, sample_index):
    X_example = model.X.iloc[sample_index]
    y_example = model.y.iloc[sample_index]
    # Get probs and convert from map to list 
    ps = list(model.get_class_probabilities(X_example).values())
    r = np.hstack((ps, y_example.values)) # combine w/ label
    model.plot_example_output(r, sample_index, priors)

def plot_new_samples(model, num_samples, indices, priors):
    """
    Randomly sample from this list and plot examples
    :param num_samples: The number of samples to randomly sample
    :param indices: all indices for a particular class to sample from
    :param priors: Boolean
    """
    rand_indices = np.random.choice(indices, num_samples, replace=False)
    for sample_index in rand_indices:
        plot_sample(model, priors, sample_index)
    return rand_indices


In [ ]:
# Plot examples for each class, first WITHOUT prior
class_indices_map = {}
for cur_class in model.class_labels:
    indices = []
    print("\nSampling class: " + str(cur_class))
    for index, row in model.y.iterrows():
        if cur_class in row['transient_type']:
            indices.append(index)
    rand_indices = plot_new_samples(model=model, 
                  num_samples=10, 
                  indices=indices, 
                  priors=False) 
    class_indices_map[cur_class] = rand_indices
    
# Run on same indices WITH priors
model.model.clf.class_priors = [0.65, 0.36, 0.01, 0.005]
model.class_priors = [0.65, 0.36, 0.01, 0.005]
for class_name in class_indices_map.keys():
    for index in class_indices_map[class_name]:
        plot_sample(model, 
                    priors=True, 
                    sample_index=index)